In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

sys.path.append('..')
import helper
from deepomics import neuralnetwork as nn
from deepomics import explore
np.random.seed(247)
tf.set_random_seed(247)

In [2]:

data_path = '../../data/RNAcompete_2013/rnacompete2013.h5'
experiments = helper.get_experiments_hdf5(data_path)


In [3]:
rbp_index = 0
ss_type = 'seq'
normalize_method = 'log_norm'

# load rbp dataset
train, valid, test = helper.load_dataset_hdf5(data_path, dataset_name=None, ss_type=ss_type, rbp_index=rbp_index)

# process rbp dataset
train, valid, test = helper.process_data(train, valid, test, method=normalize_method)
 
# get shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = list(train['targets'].shape)

In [4]:


def model(input_shape, output_shape):

    # create model
    layer1 = {'layer': 'input',
            'input_shape': input_shape,
            }
    layer2 = {'layer': 'conv1d', 
            'num_filters': {'start': 32, 'bounds': [1, 200], 'scale': 32, 'multiples': 4},
            'filter_size': input_shape[1]-29, #{'start': 19, 'bounds': [5, 27], 'scale': 10, 'multiples': 2, 'offset': 1},
            'norm': 'batch',
            'padding': 'VALID',
            'activation': 'relu',
            'dropout': {'start': 0.2, 'bounds': [0.0, 0.6], 'scale': 0.2},
            'mean_pool': 10,
            }
    layer3 = {'layer': 'conv1d_residual',
            'filter_size': 5,
            'function': 'relu',
            'dropout_block': {'start': 0.1, 'bounds': [0.0, 0.6], 'scale': 0.2},
            'dropout': {'start': 0.2, 'bounds': [0.0, 0.6], 'scale': 0.2},
            'mean_pool': 10,
            }
    layer4 = {'layer': 'conv1d',
            'num_filters': {'start': 128, 'bounds': [16, 512], 'scale': 48, 'multiples': 4},
            'filter_size': 3,
            'norm': 'batch',
            'activation': 'relu',
            'dropout': {'start': 0.5, 'bounds': [0.0, 0.6], 'scale': 0.2},
            'padding': 'VALID',
            }
    layer5 = {'layer': 'dense',
            'num_units': output_shape[1],
            'activation': 'linear'
            }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer4, layer5]

    # optimization parameters
    optimization = {"objective": "squared_error",
                    "optimizer": "adam",
                    "learning_rate": 0.001, #{'start': -3, 'bounds': [-4, -1], 'scale': 1.5, 'transform': 'log'},      
                    "l2": 1e-6# {'start': -6, 'bounds': [-8, -2], 'scale': 3, 'transform': 'log'},
                    # "l1": 0, 
                    }
    return model_layers, optimization

In [5]:

# load flexible models
model_layers, optimization = model(input_shape, output_shape)

# build neural optimizer 
optimizer = explore.NeuralOptimizer(model_layers, optimization)


In [6]:
# sample new hyperparameters and compare performance on validation set
optimizer.optimize(train, valid, num_trials=50, num_epochs=20, batch_size=100, verbose=1)

---------------------------------------------------------
Running baseline model

Model layers:
input
conv1d
	mean_pool: 10
	filter_size: 12
	num_filters: 32
	dropout: 0.200
conv1d
	filter_size: 3
	num_filters: 128
	dropout: 0.200
dense
	num_units: 1

Optimization:
	learning_rate: 0.001000
	l2: 0.000001

Epoch 1 out of 10 
Epoch 2 out of 10 
Epoch 3 out of 10 
Epoch 4 out of 10 
Epoch 5 out of 10 
Epoch 6 out of 10 
Epoch 7 out of 10 
Epoch 8 out of 10 
Epoch 9 out of 10 
Epoch 10 out of 10 
  valid loss:		0.47703
  valid Pearson's R:	0.75121+/-0.00000
  valid rsquare:	0.52977+/-0.00000
  valid slope:		1.12261+/-0.00000
    loss = 0.477028959134
    took 34.7217638493 seconds

---------------------------------------------------------
trial 1 out of 20
---------------------------------------------------------

Model layers:
input
conv1d
	mean_pool: 10
	filter_size: 12
	num_filters: 48
	dropout: 0.200
conv1d
	filter_size: 3
	num_filters: 124
	dropout: 0.200
dense
	num_units: 1

Optimizat

Epoch 1 out of 10 
Epoch 2 out of 10 
Epoch 3 out of 10 
Epoch 4 out of 10 
Epoch 5 out of 10 
Epoch 6 out of 10 
Epoch 7 out of 10 
Epoch 8 out of 10 
Epoch 9 out of 10 
Epoch 10 out of 10 
  valid loss:		0.43900
  valid Pearson's R:	0.75623+/-0.00000
  valid rsquare:	0.56365+/-0.00000
  valid slope:		1.05756+/-0.00000
Results:
loss = 0.438999375764
took 37.7741219997 seconds

---------------------------------------------------------
trial 12 out of 20
---------------------------------------------------------

Model layers:
input
conv1d
	mean_pool: 10
	filter_size: 12
	num_filters: 92
	dropout: 0.200
conv1d
	filter_size: 3
	num_filters: 100
	dropout: 0.200
dense
	num_units: 1

Optimization:
	learning_rate: 0.001000
	l2: 0.000001

Epoch 1 out of 10 
Epoch 2 out of 10 
Epoch 3 out of 10 
Epoch 4 out of 10 
Epoch 5 out of 10 
Epoch 6 out of 10 
Epoch 7 out of 10 
Epoch 8 out of 10 
Epoch 9 out of 10 
Epoch 10 out of 10 
  valid loss:		0.47197
  valid Pearson's R:	0.75146+/-0.00000
  vali

In [8]:
# print optimal model (lowest cross-validation loss)
optimizer.print_optimal_model()


Model layers:
input
conv1d
	mean_pool: 10
	filter_size: 12
	num_filters: 140
	dropout: 0.230
conv1d
	filter_size: 3
	num_filters: 68
	dropout: 0.100
dense
	num_units: 1

Optimization:
	learning_rate: 0.001000
	l2: 0.000001
